In [81]:
#-*- coding:utf-8 -*-
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torch.utils.data as data

train = pd.read_csv("data.csv",parse_dates=["Date"])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wind1 = pd.read_csv("./dataset/wind1.csv")
wind2 = pd.read_csv("./dataset/wind2.csv")
wind3 = pd.read_csv("./dataset/wind3.csv")
wind4 = pd.read_csv("./dataset/wind4.csv")

In [82]:
# Hyper-Parameters
sequence_length = 20
input_size = 2
hidden_size = 64
num_layers = 2
output_size = 4
batch_size = 10
num_epochs = 50
learning_rate = 5e-5

In [83]:
class mydata(data.Dataset):
    def __init__(self, input_dat, y,seq_length):
        self.input_dat = torch.tensor(input_dat).float()
        self.y = torch.tensor(y).float()
        self.seq_length = seq_length

    def __getitem__(self, idx):
        if idx >= self.seq_length - 1:
            i_start = idx - self.seq_length + 1
            x = self.input_dat[i_start:(idx-3),:]
            y = self.y[(idx-3):(idx+1)]
        elif idx>=3:
            padding = self.input_dat[0].repeat(self.seq_length-idx-1,1)
            x = self.input_dat[0:(idx-3),:]
            x = torch.cat((padding,x),0)
            y = self.y[(idx-3):(idx+1)]
        else:
            x = self.input_dat[0].repeat(self.seq_length-4,1)
            padding_y = self.y[0].repeat(3-idx)
            y = self.y[0:(idx+1)]
            y = torch.cat((padding_y,y),0)
        return x, y
    
    def __len__(self):
        return self.input_dat.shape[0]

Using two strategies,

1. Test using MA data and Test relative error using MA data
2. Test using original data and test re using original data

Later, I'll figure out how to test using original data

Split the 4th dataset to test set and the first 3 dataset as training set.

In [84]:
# Training Sets
train1 = np.array(wind1[["GridPowerMA","WindSpdMA"]][19:])
train_data1 = mydata(train1, train1[:,1], sequence_length)
train2 = np.array(wind2[["GridPowerMA","WindSpdMA"]][19:])
train_data2 = mydata(train2, train2[:,1], sequence_length)
train3 = np.array(wind3[["GridPowerMA","WindSpdMA"]][19:])
train_data3 = mydata(train3, train3[:,1], sequence_length)

# Test Sets
test = np.array(wind4[["GridPowerMA","WindSpdMA"]][19:])
test_target = np.array(wind4["GridPower"][19:])
test_data = mydata(test, test_target, sequence_length)
test_data1 = mydata(test,test[:,1],sequence_length)

# Data Loader
train_dat1 = data.DataLoader(dataset=train_data1, batch_size=batch_size, shuffle=True)
train_dat2 = data.DataLoader(dataset=train_data2, batch_size=batch_size, shuffle=True)
train_dat3 = data.DataLoader(dataset=train_data3, batch_size=batch_size, shuffle=True)
test_dat = data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
test_dat1 = data.DataLoader(dataset=test_data1, batch_size=batch_size, shuffle=True)

In [85]:
##Build RNN(LSTM)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size,output_size)

    def forward(self, x):
        # set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

lstm_ma = RNN(input_size, hidden_size, num_layers, output_size).to(device)
lstm_o = RNN(input_size,hidden_size, num_layers, output_size).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
op_ma = torch.optim.Adam(lstm_ma.parameters(), lr=learning_rate)
op_o = torch.optim.Adam(lstm_ma.parameters(), lr=learning_rate)

In [86]:
# Training
for epoch in range(num_epochs):
    for i, (value,labels) in enumerate(train_dat1):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat2):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat3):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 500 == 0:
            print(loss.item())

0.14612074196338654
0.17041724920272827
0.03863799571990967
0.0016904885414987803
0.01225659716874361
0.02867770567536354
0.0033752568997442722
0.002689049346372485
0.0019973893649876118
0.0041188341565430164
0.0019293560180813074
0.0006379238329827785
0.006164403632283211
0.0014794185990467668
0.001406988361850381
0.0005143170128576458
0.0034693307243287563
0.0027775708585977554
0.0017191050574183464
0.0005371536826714873
0.005472785327583551
0.004536864347755909
0.001497647725045681
0.0005971401114948094
0.0016517939511686563
0.001468162750825286
0.0015611464623361826
0.0008920500986278057
0.002659273101016879
0.0032231914810836315
0.00044612440979108214
0.0004657024110201746
0.0028955754823982716
0.0007331904489547014
0.0010867404052987695
0.0005081298877485096
0.0022646277211606503
0.0007999836234375834
0.0007833450799807906
0.0004945441032759845
0.0028037051670253277
0.003067599842324853
0.002731767948716879
0.0004918597987852991
0.001753497519530356
0.0018597164889797568
0.001466

In [87]:
# Evaluation
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.0419)
0.1605526801002653
0.7101374864578247
0.15656882524490356
0.4780701754385965


In [88]:
# Evaluation using ma data
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat1:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.0004)
0.013507568038869322
0.09291854500770569
0.0100894495844841
1.0


If just input the original data, let's see what will happen.

In [89]:
# Training Sets
train1 = np.array(wind1[["GridPower","WindSpd"]][19:])
train_data1 = mydata(train1, train1[:,1], sequence_length)
train2 = np.array(wind2[["GridPower","WindSpdMA"]][19:])
train_data2 = mydata(train2, train2[:,1], sequence_length)
train3 = np.array(wind3[["GridPower","WindSpd"]][19:])
train_data3 = mydata(train3, train3[:,1], sequence_length)

# Data Loader
train_dat1 = data.DataLoader(dataset=train_data1, batch_size=batch_size, shuffle=True)
train_dat2 = data.DataLoader(dataset=train_data2, batch_size=batch_size, shuffle=True)
train_dat3 = data.DataLoader(dataset=train_data3, batch_size=batch_size, shuffle=True)
test_dat = data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
test_dat1 = data.DataLoader(dataset=test_data1, batch_size=batch_size, shuffle=True)

In [90]:
# Training
for epoch in range(num_epochs):
    for i, (value,labels) in enumerate(train_dat1):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_o(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_o.zero_grad()
        loss.backward()
        op_o.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat2):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_o(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_o.zero_grad()
        loss.backward()
        op_o.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat3):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_o.zero_grad()
        loss.backward()
        op_o.step()

        if i % 500 == 0:
            print(loss.item())

0.1531873345375061
0.12544545531272888
0.0009025622857734561
0.2077295482158661
0.07843682914972305
0.0010841478360816836
0.086118683218956
0.12809598445892334
0.0031815648544579744
0.08365408331155777
0.11831847578287125
0.000769954058341682
0.2051692008972168
0.08468858897686005
0.0014968625036999583
0.1295761913061142
0.13531726598739624
0.0007499171188101172
0.15072233974933624
0.1124248281121254
0.00595102971419692
0.16093870997428894
0.1186675876379013
0.006633176002651453
0.13628099858760834
0.09450796991586685
0.015297988429665565
0.1525070071220398
0.08831898868083954
0.013849550858139992
0.1587451696395874
0.09828118234872818
0.0028796123806387186
0.12943123281002045
0.0926542654633522
0.0030538467690348625
0.1430230438709259
0.0819411650300026
0.0010952467564493418
0.24250474572181702
0.08089534193277359
0.0019500041380524635
0.24121515452861786
0.10727778822183609
0.0010666244197636843
0.12220825999975204
0.08426358550786972
0.0014305065851658583
0.20956949889659882
0.10244

In [91]:
# Evaluation
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.0527)
0.17110022030640068
0.7066285610198975
0.14498671144247055
0.5184210526315789


In [92]:
# Hyper-Parameters
sequence_length = 20
input_size = 2
hidden_size = 64
num_layers = 2
output_size = 4
batch_size = 10
num_epochs = 500
learning_rate = 5e-5

lstm = RNN(input_size, hidden_size, num_layers, output_size).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
op = torch.optim.Adam(lstm_ma.parameters(), lr=learning_rate)

In [93]:
# Training Sets
train1 = np.array(wind1[["GridPowerMA","WindSpdMA"]][19:])
train_data1 = mydata(train1, train1[:,1], sequence_length)
train2 = np.array(wind2[["GridPowerMA","WindSpdMA"]][19:])
train_data2 = mydata(train2, train2[:,1], sequence_length)
train3 = np.array(wind3[["GridPowerMA","WindSpdMA"]][19:])
train_data3 = mydata(train3, train3[:,1], sequence_length)

# Test Sets
test = np.array(wind4[["GridPowerMA","WindSpdMA"]][19:])
test_target = np.array(wind4["GridPower"][19:])
test_data = mydata(test, test_target, sequence_length)
test_data1 = mydata(test,test[:,1],sequence_length)

# Data Loader
train_dat1 = data.DataLoader(dataset=train_data1, batch_size=batch_size, shuffle=True)
train_dat2 = data.DataLoader(dataset=train_data2, batch_size=batch_size, shuffle=True)
train_dat3 = data.DataLoader(dataset=train_data3, batch_size=batch_size, shuffle=True)
test_dat = data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
test_dat1 = data.DataLoader(dataset=test_data1, batch_size=batch_size, shuffle=True)

In [94]:
# Training
for epoch in range(num_epochs):
    for i, (value,labels) in enumerate(train_dat1):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op.zero_grad()
        loss.backward()
        op.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat2):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op.zero_grad()
        loss.backward()
        op.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat3):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op.zero_grad()
        loss.backward()
        op.step()

        if i % 500 == 0:
            print(loss.item())

0.07628988474607468
0.1154961809515953
0.06399889290332794
0.15725940465927124
0.11637923866510391
0.07082464545965195
0.15220190584659576
0.11052770912647247
0.0721173882484436
0.1282324492931366
0.10989866405725479
0.07086174190044403
0.18796448409557343
0.07573775947093964
0.0737876296043396
0.2390126883983612
0.09077264368534088
0.07214216887950897
0.18955691158771515
0.09317359328269958
0.07101200520992279
0.18718266487121582
0.11842825263738632
0.07725413888692856
0.23381051421165466
0.08929286897182465
0.09051570296287537
0.25904297828674316
0.10889600217342377
0.06825050711631775
0.16577009856700897
0.12797948718070984
0.08025266975164413
0.1806907057762146
0.1020045131444931
0.07194123417139053
0.13029657304286957
0.12416546046733856
0.06657656282186508
0.1630934178829193
0.10000459104776382
0.069209985435009
0.19660945236682892
0.14300093054771423
0.06579163670539856
0.13010874390602112
0.09650585055351257
0.0682736486196518
0.19731338322162628
0.11823345720767975
0.075024180

KeyboardInterrupt: 

In [ ]:
# Evaluation
lstm.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.6673)
0.6081427295731262
2.0059587955474854
0.48045556247234344
0.20350877192982456
